<a href="https://colab.research.google.com/github/MAPWAPS/Alien_Tree_Mapping/blob/main/04062025_SNIC_Olifantsfontein_Alien_MapPoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This notebook uses Sentinel-2 imagery to map invasive alien trees in the Olifants-Doring catchment using SNIC and the Random Forest classiifer.

### Authenticate GEE and load the required datasets

In [ ]:
########Import necessary python libraries###################################################################################
import ee
import geemap
import datetime

####Authenticate and initialize Earth Engine############################################################################################
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='thandeka-skosana')

#############Define the paths of you=r training data as well as AOI, exlc B1, B9, B10####################################
trainDAT = ee.FeatureCollection("projects/olifantsdoringproject/assets/06052025_trainingdata")
aoi = ee.FeatureCollection("projects/olifantsdoringproject/assets/Olifants_Doring_Catchment")

############Compute Sentinel-2 Median Composite for you time period using cloud-free scenes#############
image = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(aoi) \
    .filterDate("2025-05-08", "2025-05-21") \
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)) \
    .median() \
    .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A', 'B11', 'B12']) \
    .clip(aoi)


In [ ]:
###########Adding ALOS datasets###########
# ALOS Landform (JAXA)
landform = ee.Image('CSP/ERGo/1_0/Global/ALOS_landforms').select('constant')

# ALOS DSM Elevation
elevation = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').mosaic().select('DSM')

terrain_bands = [elevation, landform]

#add biome
biomes = ee.Image('projects/olifantsdoringproject/assets/Biomes_OD')
biome_band= biomes.select('b1').rename('biome');

/usr/local/lib/python3.11/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for JAXA/ALOS/AW3D30/V3_2! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/JAXA_ALOS_AW3D30_V3_2

  warnings.warn(warning, category=DeprecationWarning)


### Calculate indices

In [ ]:
def add_indices(image):
    # 1. Normalized Difference Vegetation Index (NDVI)
    NDVI = image.expression('(NIR - RED) / (NIR + RED)', {
        'NIR': image.select('B8'),
        'RED': image.select('B4')
    }).rename('NDVI')

    # 2. Chlorophyll Green Index (Chlogreen)
    Chlogreen = image.expression('(NIRnarrow) / (Green + REDedge1)', {
        'NIRnarrow': image.select('B8A'),
        'Green': image.select('B3'),
        'REDedge1': image.select('B5')
    }).rename('Chlogreen')

    # 3. Leaf Anthocyanin Content Index (LAnthoC)
    LAnthoC = image.expression('(REDedge3) / (Green + REDedge1)', {
        'REDedge3': image.select('B7'),
        'Green': image.select('B3'),
        'REDedge1': image.select('B5')
    }).rename('LAnthoC')

    # 4. Leaf Chlorophyll Content Index (LChloC)
    LChloC = image.expression('(REDedge3) / (REDedge1)', {
        'REDedge3': image.select('B7'),
        'REDedge1': image.select('B5')
    }).rename('LChloC')

    # 5. Leaf Carotenoid Content Index (LCaroC)
    LCaroC = image.expression('(REDedge3) / (Blue - REDedge1)', {
        'REDedge3': image.select('B7'),
        'Blue': image.select('B2'),
        'REDedge1': image.select('B5')
    }).rename('LCaroC')

    # 6. Bare Soil Index (BAI)
    BAI = image.expression('(Blue - NIR) / (Blue + NIR)', {
        'Blue': image.select('B2'),
        'NIR': image.select('B8')
    }).rename('BAI')

    # 7. Greenness Index (GI)
    GI = image.expression('(Green / Red)', {
        'Green': image.select('B3'),
        'Red': image.select('B4')
    }).rename('GI')

    # 8. Green NDVI (gNDVI)
    gNDVI = image.expression('(NIR - Green) / (NIR + Green)', {
        'Green': image.select('B3'),
        'NIR': image.select('B8')
    }).rename('gNDVI')

    # 9. Moisture Stress Index (MSI)
    MSI = image.expression('(SWIR1 / NIR)', {
        'SWIR1': image.select('B11'),
        'NIR': image.select('B8')
    }).rename('MSI')

    # 10. Normalized Difference Red Edge and SWIR (NDrededgeSWIR)
    NDrededgeSWIR = image.expression('(Rededge2 - SWIR2) / (Rededge2 + SWIR2)', {
        'Rededge2': image.select('B6'),
        'SWIR2': image.select('B12')
    }).rename('NDrededgeSWIR')

    # 11. Normalized Difference Tillage Index (NDTI)
    NDTI = image.expression('(SWIR1 - SWIR2) / (SWIR1 + SWIR2)', {
        'SWIR1': image.select('B11'),
        'SWIR2': image.select('B12')
    }).rename('NDTI')

    # 12. Red Edge NDVI (NDVIre)
    NDVIre = image.expression('(NIR - Rededge1) / (NIR + Rededge1)', {
        'NIR': image.select('B8'),
        'Rededge1': image.select('B5')
    }).rename('NDVIre')

    # 13. NDVI Variant 1 (NDVI1)
    NDVI1 = image.expression('(NIR - SWIR1) / (NIR + SWIR1)', {
        'NIR': image.select('B8'),
        'SWIR1': image.select('B11')
    }).rename('NDVI1')

    # 14. NDVI Variant 2 (NDVI2)
    NDVI2 = image.expression('(Green - NIR) / (Green + NIR)', {
        'NIR': image.select('B8'),
        'Green': image.select('B3')
    }).rename('NDVI2')

    # 15. Enhanced Vegetation Index (EVI)
    EVI = image.expression('2.5 * ((NIR - Red) / (NIR + 6 * Red - 7.5 * Blue + 1))', {
        'NIR': image.select('B8'),
        'Red': image.select('B4'),
        'Blue': image.select('B2')
    }).rename('EVI')

    # 16. Enhanced Vegetation Index 2 (EVI2)
    EVI2 = image.expression('2.4 * ((NIR - Red) / (NIR + Red + 1))', {
        'NIR': image.select('B8'),
        'Red': image.select('B4')
    }).rename('EVI2')

    # 17. Normalized Green Reflectance (Norm_G)
    Norm_G = image.expression('(Green) / (NIRwide + Red + Green)', {
        'NIRwide': image.select('B8'),
        'Green': image.select('B3'),
        'Red': image.select('B4')
    }).rename('Norm_G')

    # 18. Normalized NIR Reflectance (Norm_NIR)
    Norm_NIR = image.expression('(NIRwide) / (NIRwide + Red + Green)', {
        'NIRwide': image.select('B8'),
        'Green': image.select('B3'),
        'Red': image.select('B4')
    }).rename('Norm_NIR')

    # 19. Normalized Red Reflectance (Norm_Red)
    Norm_Red = image.expression('(Red) / (NIRwide + Red + Green)', {
        'NIRwide': image.select('B8'),
        'Green': image.select('B3'),
        'Red': image.select('B4')
    }).rename('Norm_Red')

    # 20. Red Edge Peak Area
    RededgePeakArea = image.expression('(Red + Rededge1 + Rededge2 + Rededge3 + NIRnarrow)', {
        'NIRnarrow': image.select('B8A'),
        'Rededge1': image.select('B5'),
        'Rededge2': image.select('B6'),
        'Rededge3': image.select('B7'),
        'Red': image.select('B4')
    }).rename('RededgePeakArea')

    # 21. Red - SWIR1 Index (RedSWIR1)
    RedSWIR1 = image.expression('(Red - SWIR)', {
        'SWIR': image.select('B11'),
        'Red': image.select('B4')
    }).rename('RedSWIR1')

    # 22. Simple Ratio Blue / Red Edge 2 (SRBlueRededge2)
    SRBlueRededge2 = image.expression('(Blue / Rededge2)', {
        'Blue': image.select('B2'),
        'Rededge2': image.select('B6')
    }).rename('SRBlueRededge2')

    # 23. Simple Ratio Blue / Red Edge 3 (SRBlueRededge3)
    SRBlueRededge3 = image.expression('(Blue / Rededge3)', {
        'Blue': image.select('B2'),
        'Rededge3': image.select('B7')
    }).rename('SRBlueRededge3')

    # 24. Simple Ratio NIR Narrow / Blue (SRNIRnarrowBlue)
    SRNIRnarrowBlue = image.expression('(NIRnarrow / Blue)', {
        'NIRnarrow': image.select('B8A'),
        'Blue': image.select('B2')
    }).rename('SRNIRnarrowBlue')

    # 25. Simple Ratio NIR Narrow / Red (SRNIRnarrowRed)
    SRNIRnarrowRed = image.expression('(NIRnarrow / Red)', {
        'NIRnarrow': image.select('B8A'),
        'Red': image.select('B4')
    }).rename('SRNIRnarrowRed')

    # 26. Simple Ratio NIR Narrow / Red Edge 3 (SRNIRnarrowRededge3)
    SRNIRnarrowRededge3 = image.expression('(NIRnarrow / Rededge3)', {
        'NIRnarrow': image.select('B8A'),
        'Rededge3': image.select('B7')
    }).rename('SRNIRnarrowRededge3')

    # 27. Shortwave Trough Index (STI)
    STI = image.expression('SWIR1 / SWIR2', {
        'SWIR1': image.select('B11'),
        'SWIR2': image.select('B12')
    }).rename('STI')

    # 28. Water Band Index (WBI)
    WBI = image.expression('(Blue - Red) / (Blue + Red)', {
        'Blue': image.select('B2'),
        'Red': image.select('B4')
    }).rename('WBI')

    # 29. Normalized Difference Moisture Index (NDMI)
    NDMI = image.expression('(NIR - SWIR) / (NIR + SWIR)', {
        'NIR': image.select('B8'),
        'SWIR': image.select('B11')
    }).rename('NDMI')

    # Return image with all added indices
    return image.addBands([
        NDVI, Chlogreen, LAnthoC, LChloC, LCaroC, BAI, GI, gNDVI, MSI,
        NDrededgeSWIR, NDTI, NDVIre, NDVI1, NDVI2, EVI, EVI2,
        Norm_G, Norm_NIR, Norm_Red, RededgePeakArea, RedSWIR1,
        SRBlueRededge2, SRBlueRededge3, SRNIRnarrowBlue, SRNIRnarrowRed,
        SRNIRnarrowRededge3, STI, WBI, NDMI
    ])



### Apply SNIC

In [ ]:
# load image with bands and indices
snic_predictors = ["B2", "B3", "B4"]
selected_image = image.select(snic_predictors)

# Run SNIC segmentation
snic = ee.Algorithms.Image.Segmentation.SNIC(
    image=selected_image,
    size=10,
    compactness=0,
    connectivity=8,
    neighborhoodSize=40,
    seeds=None
)
# Extract the 'clusters' band and rename it to 'segments'
segments = snic.select("clusters").rename("segments")

# Apply the add_indices function to the single image
image_indices = add_indices(image).addBands(terrain_bands).addBands(biome_band).addBands(segments)
image_indices


### Export SNIC segments as an asset to GEE

In [ ]:
# Compute segment boundaries
#edges = segments.fastDistanceTransform(30).sqrt().mask().rename('edges')

# Convert the FeatureCollection to a Geometry
#ROI = aoi.geometry()
#task = ee.batch.Export.image.toAsset(
    #image=edges.visualize(palette=['000000'], forceRgbOutput=True),
    #description='edges_Segmented_Image_Exportclassified_export',
    #assetId='projects/olifantsdoringproject/assets/snic_edgessegments02062025',  # Replace with your actual asset path
    #region=aoi.geometry(),  # Ensure 'aoi' is defined as ee.FeatureCollection or ee.Geometry
    #scale=10,
    #maxPixels=1e13
#)
#task.start()

### Visualize SNIC segments

In [ ]:
#########Visualize your segments###################################################################
Map = geemap.Map(zoom=11, basemap='SATELLITE')
Map.centerObject(aoi, 11)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel-2 RGB')
Map.addLayer(segments.randomVisualizer(), {}, 'SNIC Segments')
Map.addLayer(image_indices, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel-2 RGB_clipped')
Map

Map(center=[-31.69446269213873, 19.289755343477697], controls=(WidgetControl(options=['position', 'transparent…

### Split the training dataset (70-20-10) and train the classifer

In [ ]:
predictors = ["B2", "B3", "B4", "B5", "B6", "B7","B8","B8A", "B11", "B12", "NDVI", "Chlogreen", "LAnthoC", "LChloC", "LCaroC", "BAI", "GI", "gNDVI", "MSI",
        "NDrededgeSWIR", "NDTI", "NDVIre", "NDVI1", "NDVI2", "EVI", "EVI2","Norm_G", "Norm_NIR", "Norm_Red", "RededgePeakArea", "RedSWIR1", "SRBlueRededge2",
        "SRBlueRededge3", "SRNIRnarrowBlue", "SRNIRnarrowRed","SRNIRnarrowRededge3", "STI", "WBI", "NDMI", "biome", "segments"]

#########Extract unique classes from your training data##########################################
classes = trainDAT.aggregate_array("Id").distinct().getInfo()

#########Split training into 90% working, 10% test################################################
held_out_test, working_set = [], []

for class_value in classes:
    class_fc = trainDAT.filter(ee.Filter.eq("Id", class_value)).randomColumn("rand_holdout", seed=999)
    held_out_test.append(class_fc.filter(ee.Filter.gte("rand_holdout", 0.9)))
    working_set.append(class_fc.filter(ee.Filter.lt("rand_holdout", 0.9)))

testSet_fixed = ee.FeatureCollection(held_out_test).flatten()
working_data = ee.FeatureCollection(working_set).flatten()

### Export working and test subset to check

# Export test set to Google Drive
#test_export = ee.batch.Export.table.toDrive(
    #collection=testSet_fixed,
    #description='TestSet_fixed_Export',
    #fileFormat='CSV'
#)
#test_export.start()

# Export working set to Google Drive
#working_export = ee.batch.Export.table.toDrive(
    #collection=working_data,
    #description='WorkingData_Export',
    #fileFormat='CSV'
#)
#working_export.start()

#print("Export started. Go to https://code.earthengine.google.com/tasks to monitor the progress.")

################ Run x number of iterations ##################################################################
for i in range(50):
    print(f"\n=== Iteration {i + 1} ===")
    trainList, valList = [], []

    for class_value in classes:
        class_fc = working_data.filter(ee.Filter.eq("Id", class_value)).randomColumn("rand_iter", seed=i + 200)
        trainList.append(class_fc.filter(ee.Filter.lt("rand_iter", 0.7778)))  # 70% of 90%
        valList.append(class_fc.filter(ee.Filter.gte("rand_iter", 0.7778)))  # 20% of 90%

    trainSet = ee.FeatureCollection(trainList).flatten()
    valSet = ee.FeatureCollection(valList).flatten()

    training = image_indices.sampleRegions(
        collection=trainSet,
        properties=["Id"],
        scale=10,
        #tileScale= 2
    )

    classifier = ee.Classifier.smileRandomForest(50).train(
        features=training,
        classProperty="Id",
        inputProperties=predictors
    )

    val = image_indices.sampleRegions(
        collection=valSet,
        properties=["Id"],
        scale=10,
        #tileScale= 2
    ).classify(classifier)

    val_matrix = val.errorMatrix("Id", "classification")
    print("Validation Accuracy:", val_matrix.accuracy().getInfo())

###########Final test on 10% held-out samples#######################################################
final_test_sample = image_indices.sampleRegions(
    collection=testSet_fixed,
    properties=["Id"],
    scale=10,
    #tileScale= 2
).classify(classifier)

test_matrix = final_test_sample.errorMatrix("Id", "classification")




=== Iteration 1 ===
Validation Accuracy: 0.9606017191977078

=== Iteration 2 ===
Validation Accuracy: 0.965212784743069

=== Iteration 3 ===
Validation Accuracy: 0.9577114427860697

=== Iteration 4 ===
Validation Accuracy: 0.9608515244452462

=== Iteration 5 ===
Validation Accuracy: 0.9607633315498484

=== Iteration 6 ===
Validation Accuracy: 0.9632660483724313

=== Iteration 7 ===
Validation Accuracy: 0.9609704641350211

=== Iteration 8 ===
Validation Accuracy: 0.965425057921939

=== Iteration 9 ===
Validation Accuracy: 0.9585953878406709

=== Iteration 10 ===
Validation Accuracy: 0.9610662925289372

=== Iteration 11 ===
Validation Accuracy: 0.9609834313201496

=== Iteration 12 ===
Validation Accuracy: 0.9623889952986244

=== Iteration 13 ===
Validation Accuracy: 0.9610718812239725

=== Iteration 14 ===
Validation Accuracy: 0.9648691375373265

=== Iteration 15 ===
Validation Accuracy: 0.9653261840929401

=== Iteration 16 ===
Validation Accuracy: 0.9629695567028663

=== Iteration 17 =

Validation Accuracy: 0.9586631486367634

=== Iteration 43 ===
Validation Accuracy: 0.9643864598025388

=== Iteration 44 ===
Validation Accuracy: 0.9608323133414932

=== Iteration 45 ===
Validation Accuracy: 0.9618425575799243

=== Iteration 46 ===
Validation Accuracy: 0.9635204533380556

=== Iteration 47 ===
Validation Accuracy: 0.9585152838427947

=== Iteration 48 ===
Validation Accuracy: 0.9596963672510392

=== Iteration 49 ===
Validation Accuracy: 0.9633783073418609

=== Iteration 50 ===
Validation Accuracy: 0.963707525351361


### Overall Accuracy

In [ ]:
print("The overall accuracy on the test dataset (held-out 10%) is:", test_matrix.accuracy().getInfo());
print("The Kappa coefficient is:", test_matrix.kappa().getInfo());


The overall accuracy on the test dataset (held-out 10%) is: 0.9602577873254565
The Kappa coefficient is: 0.9536325388085554


### Accuracy of Alien vs Non-Alien Classes

In [ ]:
# Remap Ids: 1 = Alien (1–5), 0 = Non-Alien (7–23)
def remap_binary(feature):
    id_val = feature.getNumber('Id')
    class_val = feature.getNumber('classification')

    binary_id = ee.Number(id_val).lte(5).int()
    binary_class = ee.Number(class_val).lte(5).int()

    return feature.set('binary_id', binary_id).set('binary_classification', binary_class)

# Apply the remapping
binary_test_sample = final_test_sample.map(remap_binary)

# Compute confusion matrix
binary_matrix = binary_test_sample.errorMatrix('binary_id', 'binary_classification')

# Print overall accuracy and kappa coefficient
print('Overall Accuracy (Alien vs Non-Alien):', binary_matrix.accuracy().getInfo())
print('Kappa Coefficient (Alien vs Non-Alien):', binary_matrix.kappa().getInfo())
#print('Confusion Matrix:')
#print(binary_matrix.getInfo())


Overall Accuracy (Alien vs Non-Alien): 0.9853204439670605
Kappa Coefficient (Alien vs Non-Alien): 0.948562126528194


### Accuracy between the alien classes

In [ ]:
# Filter to include only alien classes 1–5
filtered_aliens = final_test_sample.filter(ee.Filter.lte('Id', 5))

# Compute the confusion matrix using original classes (1–5)
alien_matrix = filtered_aliens.errorMatrix('Id', 'classification')

# Print accuracy metrics
print('Overall Accuracy (Alien classes 1–5):', alien_matrix.accuracy().getInfo())
print('Kappa Coefficient (Alien classes 1–5):', alien_matrix.kappa().getInfo())
print('Confusion Matrix (Alien classes 1–5):')
print(alien_matrix.getInfo())

Overall Accuracy (Alien classes 1–5): 0.9127789046653144
Kappa Coefficient (Alien classes 1–5): 0.8626660879367198
Confusion Matrix (Alien classes 1–5):
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 117, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 4], [0, 0, 33, 1, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 10], [0, 0, 1, 37, 5, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 1, 5], [0, 0, 0, 0, 260, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3], [0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

### Apply the classifier and visualize the results

In [ ]:
########Classify the image###################################################################
classified = image_indices.classify(classifier)

#########Define a color palette for the 23 classes###########################################
palette = [
    '#ff7f00', '#9900FF', '#F91DF9', '#fd0618', '#741b47', '#14870e', '#6aa84f', '#C7E88C',
    '#a8a800', '#526529', '#90EE90','#00FF00','#08f3e4', '#4d9592', '#26D4A2', '#ffff00', '#CC5500', '#ffffff',
    '#0a14f9', '#000000','#783f04', '#b7b7b7','#828282'
]

#########Define class names for legend#######################################################
class_names = [
    "Prosopis", "Wattle", "Gum", "Pine", "Other", "Riparian trees", "Succulent karoo","Riparian bush",
    "Fynbos high density", "Fynbos low density", "Renosterveld","Bushmanland shrubland", "Wetland palmiet", "Wetland reeds", "Wetland other", "Agriculture irrigated", "Agriculture dryland"
    , "Bare ground", "Water","Urban","Rock", "Shade", "Burnt"
]

#########Visualize the classification results#################################################
Map = geemap.Map(zoom=11, basemap='SATELLITE')
Map.centerObject(aoi, 11)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel-2 RGB')
Map.addLayer(classified, {'min': 1, 'max': 23, 'palette': palette}, 'Olifantsfontein Land Cover')

#########Add a legend########################################################################
def add_legend(map_object, class_names, palette):
    legend_dict = {name: color for name, color in zip(class_names, palette)}
    map_object.add_legend(title="Land Cover Classification", legend_dict=legend_dict)

add_legend(Map, class_names, palette)
#######Display the map######################################################################
Map


Map(center=[-31.69446269213873, 19.289755343477697], controls=(WidgetControl(options=['position', 'transparent…

### Export the classification as an Asset to GEE

In [ ]:
# Convert the FeatureCollection to a Geometry
ROI = aoi.geometry()
task = ee.batch.Export.image.toAsset(
    image=classified,
    description='classified_export',
    assetId='projects/olifantsdoringproject/assets/20250605_oli_class',  # Replace with your actual asset path
    region=aoi.geometry(),  # Ensure 'aoi' is defined as ee.FeatureCollection or ee.Geometry
    scale=10,
    maxPixels=1e13
)
task.start()